<a href="https://colab.research.google.com/github/DongUk-Park/RnD/blob/main/POI/NAIS/Foursquare_DataPreprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
import csv
import random

In [28]:
# 파일 경로
file_path = 'Foursquare_checkins.txt'

# 파일 읽어오기
df = pd.read_csv(file_path, sep='\t', header=None, names=['user_id', 'business_id', 'Time_stamp'])
# 'Time_stamp' 열 삭제
print(df)
df.drop('Time_stamp', axis=1, inplace=True)
data = df.values.tolist()

         user_id  business_id    Time_stamp
0              0            0  1.333447e+09
1              1            1  1.333447e+09
2              2            2  1.333447e+09
3              3            3  1.333447e+09
4              4            4  1.333447e+09
...          ...          ...           ...
1196243    23986         4137  1.379345e+09
1196244    10425         6898  1.379345e+09
1196245    13985        26509  1.379345e+09
1196246      131         2205  1.379345e+09
1196247     6792        26144  1.379345e+09

[1196248 rows x 3 columns]


In [29]:
"""
train에 사용될 history 데이터 (2차원 리스트)
인덱스 번호 : user_Id
인덱스 value(list형식) : 해당 uesr가 방문한 business_id 리스트
"""

data.sort(key=lambda x: x[0])

history_list = []
tmp = []
before_user_id = 0
for idx, i in enumerate(data):
  if i[0] == before_user_id:
    tmp.append(i[1])
  else:
    tmp = list(dict.fromkeys(tmp)) # dictionary key 특성을 이용한 중복제거
    if len(tmp) >= 10: # 중복이 제거된 방문 횟수가 10회가 넘는 유저만 append
      history_list.append(tmp)

    tmp = []
    tmp.append(i[1])
    before_user_id += 1

  if idx == len(data) - 1:
    if len(tmp) >= 10:
      history_list.append(tmp)

output_file = '/content/history_list.csv'
with open(output_file, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(history_list)

In [34]:
print(len(history_list))
print(history_list[:2])

24941
[[0, 249, 1323, 10155, 3093, 3316, 1449, 13890, 336, 4226, 163, 4684, 348, 3438, 19553, 7220, 6476, 21614, 7784, 98, 1018, 16557, 21960, 15744, 13610], [1, 1684, 1705, 1868, 2452, 1742, 4941, 5727, 8525, 6892, 9176, 1607, 9189, 9190, 1812, 9191, 3882, 740, 3989, 11873, 8153, 11880, 11881, 11884, 887, 3484, 13052, 2239, 2663, 17112, 17071, 1254, 3451, 21098, 1776, 3478, 21108, 21110, 1525, 18389, 21171, 21177, 18392, 21380, 21266, 21253, 21182, 21331, 21336, 21279, 21289, 21286, 21386, 21358, 21375, 21262, 21382, 21264, 21302, 19141, 21431, 684, 1475, 75, 13935, 4114, 4645, 3504, 9927, 14858, 19073, 12689, 5133, 2466]]


In [30]:
business_info = pd.read_csv('/content/Foursquare_poi_coos.txt', sep='\t', header=None, names = ['business_id','latitude','longitude'])
print(business_info)
business_id_list = business_info['business_id'].tolist()
latitude_list = business_info['latitude'].tolist()
longitude_list = business_info['longitude'].tolist()

       business_id   latitude   longitude
0                0  40.748939  -73.992280
1                1  40.719726  -74.002472
2                2  25.696201 -100.351632
3                3  41.875497  -87.649484
4                4  40.443403  -79.954869
...            ...        ...         ...
28588        28588  33.761346  -84.386458
28589        28589  39.963072  -75.176862
28590        28590  47.621870 -122.351738
28591        28591  40.772144  -73.984377
28592        28592  41.902165  -87.704329

[28593 rows x 3 columns]


In [36]:
"""
위 history_list를 train, validation, test 데이터셋으로 분리시킴
"""
train_ratio = 0.6
test_ratio = 0.2
validation_ratio = 0.2

train_dataset = []
test_dataset = []
validation_dataset = []

train_negative_dataset = []
test_negative_dataset = []
validation_negative_dataset = []


#아래의 결과를 csv 파일로 저장할 경로, 총 6개의 파일 생성
output_train = '/content/history_train.csv'
output_validation = '/content/history_validation.csv'
output_test = '/content/history_test.csv'

output_train_negative = '/content/history_train_negative.csv'
output_validation_negative = '/content/history_validation_negative.csv'
output_test_negative = '/content/history_test_negative.csv'


for user, history in enumerate(history_list):
  negative_list = [num for num in business_id_list if num not in history]

  ### 한 유저에 대해 train,validation,test dataset 생성
  total_samples = len(history) # 방문 횟수 저장
  random.shuffle(history) # 방문 기록 순서를 랜덤하게 섞어준다.

  train_size = int(total_samples * train_ratio)
  validation_size = int(total_samples * validation_ratio)

  list_train = history[:train_size]
  list_validation = history[train_size: (train_size + validation_size)]
  list_test = history[(train_size + validation_size):]


  train_dataset.append(list_train)
  validation_dataset.append(list_validation)
  test_dataset.append(list_test)

  total_samples = len(negative_list) # 방문 횟수 저장
  random.shuffle(negative_list) # 방문 기록 순서를 랜덤하게 섞어준다.

  train_size = int(total_samples * train_ratio)
  validation_size = int(total_samples * validation_ratio)

  negative_list_train = negative_list[:train_size]
  negative_list_validation = negative_list[train_size: (train_size + validation_size)]
  negative_list_test = negative_list[(train_size + validation_size):]

  train_negative_dataset.append(negative_list_train)
  validation_negative_dataset.append(negative_list_validation)
  test_negative_dataset.append(negative_list_test)

with open(output_train, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(train_dataset)
with open(output_validation, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(validation_dataset)
with open(output_test, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(test_dataset)

with open(output_train_negative, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(train_negative_dataset)
with open(output_validation_negative, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(validation_negative_dataset)
with open(output_test_negative, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(test_negative_dataset)